In [1]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets
import pandas as pd
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [3]:
import tensorflow
tensorflow.keras.__version__

'2.2.4-tf'

In [9]:
# Read in CSV files
employment = pd.read_csv('../COVID-19-Predictive-Modelling/assets/data/AGS_data/AGS_Data_Massaged.csv')
covid = pd.read_csv('../COVID-19-Predictive-Modelling/assets/data/COVID County Data/covid_county_data_cleaned.csv')
social = pd.read_csv('../COVID-19-Predictive-Modelling/assets/data/SVI2018_US_COUNTY.csv')

In [11]:
employment.head()

,fips,county_name,labor_force,file_week_ended,percent_unemployed,total_unemployed
0,1001,Autauga,25819,3/7/20,2.69,694
1,1001,Autauga,25819,3/14/20,2.78,718
2,1001,Autauga,25819,3/21/20,3.34,862
3,1001,Autauga,25819,3/28/20,7.47,1929
4,1001,Autauga,25819,4/4/20,12.92,3336


In [12]:
covid.head()

,Unnamed: 0,name,date,fips,lat,long,confirmed,deaths,confirmed_diff,deaths_diff,last_update,state,state_abbr
0,0,Jefferson,2020-03-22,1073.0,33.555547,-86.895063,71,0,0,0,2020-03-22 23:45:00,Alabama,AL
1,1,Shelby,2020-03-22,1117.0,33.268798,-86.662326,17,0,0,0,2020-03-22 23:45:00,Alabama,AL
2,2,Lee,2020-03-22,1081.0,32.601549,-85.351322,16,0,0,0,2020-03-22 23:45:00,Alabama,AL
3,3,Madison,2020-03-22,1089.0,34.763271,-86.550696,16,0,0,0,2020-03-22 23:45:00,Alabama,AL
4,4,Tuscaloosa,2020-03-22,1125.0,33.287261,-87.525568,7,0,0,0,2020-03-22 23:45:00,Alabama,AL


In [13]:
social.head()

,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,...,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,...,0,0,0,1,-999,4160,588,10.6,1.5,32290
1,1,ALABAMA,AL,Autauga,1001,"Autauga County, Alabama",594.443459,55200,0,23315,...,0,0,0,0,0,3875,508,7.1,0.9,37301
2,1,ALABAMA,AL,Blount,1009,"Blount County, Alabama",644.830460,57645,0,24222,...,0,0,0,0,0,6303,732,11.0,1.3,40036
3,1,ALABAMA,AL,Butler,1013,"Butler County, Alabama",776.838201,20025,0,10026,...,0,0,0,0,0,2005,340,10.2,1.7,17280
4,1,ALABAMA,AL,Calhoun,1015,"Calhoun County, Alabama",605.867251,115098,0,53682,...,0,0,0,0,0,10686,796,9.4,0.7,117894


In [14]:
# Rename fips to match across csvs
social.rename(columns={'FIPS':'fips'}, inplace=True)
social.head()

,ST,STATE,ST_ABBR,COUNTY,fips,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,...,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,...,0,0,0,1,-999,4160,588,10.6,1.5,32290
1,1,ALABAMA,AL,Autauga,1001,"Autauga County, Alabama",594.443459,55200,0,23315,...,0,0,0,0,0,3875,508,7.1,0.9,37301
2,1,ALABAMA,AL,Blount,1009,"Blount County, Alabama",644.830460,57645,0,24222,...,0,0,0,0,0,6303,732,11.0,1.3,40036
3,1,ALABAMA,AL,Butler,1013,"Butler County, Alabama",776.838201,20025,0,10026,...,0,0,0,0,0,2005,340,10.2,1.7,17280
4,1,ALABAMA,AL,Calhoun,1015,"Calhoun County, Alabama",605.867251,115098,0,53682,...,0,0,0,0,0,10686,796,9.4,0.7,117894


In [18]:
# Merge social and employment on fips
merge = pd.merge(social, employment, on='fips')
merge.head()

,ST,STATE,ST_ABBR,COUNTY,fips,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,...,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,county_name,labor_force,file_week_ended,percent_unemployed,total_unemployed
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,...,4160,588,10.6,1.5,32290,Rio-Arriba,14431,3/7/20,4.89,706
1,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,...,4160,588,10.6,1.5,32290,Rio-Arriba,14431,3/14/20,5.00,721
2,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,...,4160,588,10.6,1.5,32290,Rio-Arriba,14431,3/21/20,7.21,1040
3,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,...,4160,588,10.6,1.5,32290,Rio-Arriba,14431,3/28/20,10.61,1531
4,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,...,4160,588,10.6,1.5,32290,Rio-Arriba,14431,4/4/20,13.80,1992


In [20]:
# Merge merge and covid on fips
combined = pd.merge(merge, covid, on='fips')
combined.head()

,ST,STATE,ST_ABBR,COUNTY,fips,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,...,date,lat,long,confirmed,deaths,confirmed_diff,deaths_diff,last_update,state,state_abbr
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,...,2020-03-22,36.510721,-106.693167,0,0,0,0,2020-03-22 23:45:00,New Mexico,NM
1,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,...,2020-03-23,36.510721,-106.693167,0,0,0,0,2020-03-23 23:19:34,New Mexico,NM
2,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,...,2020-03-24,36.510721,-106.693167,0,0,0,0,2020-03-24 23:37:31,New Mexico,NM
3,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,...,2020-03-25,36.510721,-106.693167,2,0,2,0,2020-03-25 23:33:19,New Mexico,NM
4,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,...,2020-03-26,36.510721,-106.693167,2,0,0,0,2020-03-26 23:48:35,New Mexico,NM


In [ ]:
# Data Preprocessing

In [ ]:
# Train

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [ ]:
# Scale